In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pyp
import seaborn as sns
%matplotlib inline

In [27]:
import warnings
warnings.filterwarnings("ignore")

In [28]:
legitimate_url_df = pd.read_csv("../Input/data_legitimate_36400.csv")
col_names = ["index", "url"]
legitimate_url_df.columns = col_names

legitimate_url_df = list(legitimate_url_df["url"])
first_50_legitimate_url_df = legitimate_url_df[:50]
print(first_50_legitimate_url_df)


['http://www.conceptdraw.com/How-To-Guide/Local-Area-Network-gogle-facebook-admin-loginsecure', 'https://www.edrawsoft.com/Local-Area-Network.php', 'http://www.webopedia.com/TERM/L/local_area_network_LAN.html', 'https://www.acsac.org/secshelf/book001/16.pdf', 'http://www.diffen.com/difference/LAN_vs_WAN', 'http://sentence.yourdictionary.com/lan', 'http://www.itrelease.com/2012/12/examples-and-types-of-networks/', 'http://www.dictionary.com/browse/lan', 'http://fcit.usf.edu/Network/chap1/chap1.htm', 'http://www.Cisco.com/c/en/us/support/wireless/2100-series-wireless-lan-controllers/products-configuration-examples-list.html', 'http://support.dlink.com/emulators/dcw725/RgConnect.html', 'http://www.maxis.com.my/content/dam/maxis/en/personal/support/faq-images/devices/all-internet-devices/fibre/user-manual/fiber%20internet-%20MB%20TG784%20User%20Manual.pdf', 'http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.178.31&rep=rep1&type=pdf', 'http://www.Cisco.com/c/dam/en/us/td/docs/video/a

In [29]:
import domain_parser

parsed_urls = domain_parser.parse(first_50_legitimate_url_df, "legitimate")
print(parsed_urls)

[{'url': 'http://www.conceptdraw.com/How-To-Guide/Local-Area-Network-gogle-facebook-admin-loginsecure', 'domain': 'conceptdraw', 'registered_domain': 'conceptdraw.com', 'tld': 'com', 'subdomain': 'www', 'class': 'legitimate', 'id': 0, 'protocol': 'http', 'path': '/How-To-Guide/Local-Area-Network-gogle-facebook-admin-loginsecure', 'words_raw': ['conceptdraw', 'how', 'to', 'guide', 'local', 'area', 'network', 'gogle', 'facebook', 'admin', 'loginsecure', 'www']}, {'url': 'https://www.edrawsoft.com/Local-Area-Network.php', 'domain': 'edrawsoft', 'registered_domain': 'edrawsoft.com', 'tld': 'com', 'subdomain': 'www', 'class': 'legitimate', 'id': 1, 'protocol': 'https', 'path': '/Local-Area-Network.php', 'words_raw': ['edrawsoft', 'local', 'area', 'network', 'php', 'www']}, {'url': 'http://www.webopedia.com/TERM/L/local_area_network_LAN.html', 'domain': 'webopedia', 'registered_domain': 'webopedia.com', 'tld': 'com', 'subdomain': 'www', 'class': 'legitimate', 'id': 2, 'protocol': 'http', 'pa

In [30]:
from dataPreProcessing import filter_brands
from dataPreProcessing import filter_keywords
from dataPreProcessing import subtract_lists
from dataPreProcessing import filter_randoms
from dataPreProcessing import filter_composes
from dataPreProcessing import brand_names
from dataPreProcessing import keywords
from dataPreProcessing import filter_maliciouses

for parsed_url in parsed_urls:
    parsed_url["words_raw_for_filters"] = list(parsed_url["words_raw"]) # remove from all dicts at the end
    
    # brands
    parsed_url["brand_words"] = filter_brands(parsed_url["words_raw_for_filters"])
    parsed_url["words_raw_for_filters"] = subtract_lists(parsed_url["words_raw_for_filters"], parsed_url["brand_words"])

    # keywords
    parsed_url["keyword_words"] = filter_keywords(parsed_url["words_raw_for_filters"])
    parsed_url["words_raw_for_filters"] = subtract_lists(parsed_url["words_raw_for_filters"], parsed_url["keyword_words"])

    # randoms
    parsed_url["random_words"] = filter_randoms(parsed_url["words_raw_for_filters"])
    parsed_url["words_raw_for_filters"] = subtract_lists(parsed_url["words_raw_for_filters"], parsed_url["random_words"])

    # filter word less then 7 letters
    parsed_url["word_list"] = []
    for word in list(parsed_url["words_raw_for_filters"]):
        if len(word) <= 7:
            parsed_url["word_list"].append(word)
            parsed_url["words_raw_for_filters"].remove(word)
    
    # try to decompose the rest
    for word, composed in filter_composes(parsed_url["words_raw_for_filters"]):
        for internal_word in composed:
            if word in brand_names():
                parsed_url["brand_words"].append(internal_word)
            
            elif word in keywords():
                parsed_url["keyword_words"].append(internal_word)
            
            else:
                parsed_url["word_list"].append(internal_word)
        

        parsed_url["words_raw_for_filters"].remove(word)

    
    # the rest is non brands, non keywords, non randoms, with len more then 7 chars
    # and no composed.
    for word in list(parsed_url["words_raw_for_filters"]):
        parsed_url["word_list"].append(word)
        parsed_url["words_raw_for_filters"].remove(word)
    

    # check the rest for maliciousness
    parsed_url["word_malicious"] = [mal_word for (mal_word, _, _) in filter_maliciouses(parsed_url["word_list"])]
    parsed_url["word_list"] = subtract_lists(parsed_url["word_list"], parsed_url["word_malicious"])


    # now can remove the helper list 'words_raw_for_filters'
    #parsed_url.pop('words_raw_for_filters')



print(parsed_urls)

[{'url': 'http://www.conceptdraw.com/How-To-Guide/Local-Area-Network-gogle-facebook-admin-loginsecure', 'domain': 'conceptdraw', 'registered_domain': 'conceptdraw.com', 'tld': 'com', 'subdomain': 'www', 'class': 'legitimate', 'id': 0, 'protocol': 'http', 'path': '/How-To-Guide/Local-Area-Network-gogle-facebook-admin-loginsecure', 'words_raw': ['conceptdraw', 'how', 'to', 'guide', 'local', 'area', 'network', 'gogle', 'facebook', 'admin', 'loginsecure', 'www'], 'words_raw_for_filters': [], 'brand_words': ['facebook'], 'keyword_words': ['admin'], 'random_words': ['conceptdraw', 'www'], 'word_list': ['how', 'to', 'guide', 'local', 'area', 'network', 'log', 'insecure'], 'word_malicious': ['gogle']}, {'url': 'https://www.edrawsoft.com/Local-Area-Network.php', 'domain': 'edrawsoft', 'registered_domain': 'edrawsoft.com', 'tld': 'com', 'subdomain': 'www', 'class': 'legitimate', 'id': 1, 'protocol': 'https', 'path': '/Local-Area-Network.php', 'words_raw': ['edrawsoft', 'local', 'area', 'network'